In [1]:
#STEP 1: Required Imports
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
import json
import re
import datetime


In [3]:
#STEP 2: Load Local Free LLM from Ollama
llm = Ollama(model="llama3")


/var/folders/hf/ql26b2j15blfm7plckjj9xv80000gn/T/ipykernel_18131/2894630764.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


In [19]:
example_prompt = """
You are an intent detection assistant for a financial chatbot.
Your job is to:
1. Understand what the user wants.
2. Categorize the intent into a meaningful class (e.g., FinancialSummary, RiskAssessment, RevenueQuery, InvestmentSummary, MarketVisualization, ComparativeAnalysis, ReportSummary, ForecastRequest, CreditScoreSummary, DebtRatioAnalysis, KPIRequest, EntityExtraction, AnomalyDetection).
3. Provide a short summary of the intent.

Return the result strictly in this JSON format:
{{
  "intent_class": "<meaningful_class_name>",
  "intent_description": "<description_of_user_intent>"
}}

Examples:

User: What is the total revenue in Q4 2023?
{{
  "intent_class": "RevenueQuery",
  "intent_description": "User wants revenue summary for Q4 2023"
}}

User: Summarize the investment risk factors
{{
  "intent_class": "RiskAssessment",
  "intent_description": "User wants a summary of risk factors for an investment"
}}

User: Show me how much we earned last month
{{
  "intent_class": "RevenueQuery",
  "intent_description": "User wants last month's revenue summary"
}}

User: Extract insights from this balance sheet
{{
  "intent_class": "FinancialSummary",
  "intent_description": "User wants key insights from a financial statement"
}}

User: Summarize this investment opportunity
{{
  "intent_class": "InvestmentSummary",
  "intent_description": "User wants a summary of an investment opportunity"
}}

User: Plot the market performance of our top 5 investments
{{
  "intent_class": "MarketVisualization",
  "intent_description": "User wants to visualize performance of top 5 investments"
}}

User: Compare income for 2021 and 2022
{{
  "intent_class": "ComparativeAnalysis",
  "intent_description": "User wants to compare income across two years"
}}

User: Summarize findings from the latest uploaded financial report
{{
  "intent_class": "ReportSummary",
  "intent_description": "User wants a summary of a recently uploaded financial report"
}}

User: What do we forecast for next quarter's growth?
{{
  "intent_class": "ForecastRequest",
  "intent_description": "User is asking for future performance projections"
}}

Now do the same for this query:
User: {query}
"""

intent_prompt = PromptTemplate(
    input_variables=["query"],
    template=example_prompt
)

In [21]:
#STEP 4: Use LangChain RunnableSequence
chain = intent_prompt | llm


In [23]:
# ✅ STEP 5: Parser for structured JSON output (improved to fix incomplete block edge cases)
def parse_output(output):
    try:
        json_blocks = re.findall(r'{\s*"intent_class"\s*:\s*".*?"\s*,\s*"intent_description"\s*:\s*".*?"\s*}', output, re.DOTALL)
        if json_blocks:
            return json.loads(json_blocks[-1])
        return json.loads(output)
    except Exception as e:
        return {"intent_class": "Unknown", "intent_description": output.strip()}


In [25]:
# ✅ STEP 6: Logging Detected Intents with Timestamp
log = []

# ✅ Intent Class Mapping (Standardized Set for Classification)
VALID_INTENT_CLASSES = {
    "RevenueQuery": "Questions about revenue or earnings",
    "RiskAssessment": "Requests to analyze or summarize investment risks",
    "FinancialSummary": "Summarizing financial reports, statements, or metrics",
    "InvestmentSummary": "Asking for an overview of investment opportunities",
    "MarketVisualization": "Requests to visualize trends or charts of financial data",
    "ComparativeAnalysis": "Comparison of metrics across time periods",
    "ReportSummary": "Summarizing uploaded or referenced reports",
    "ForecastRequest": "Asking for future projections or trends",
    "CreditScoreSummary": "Queries about credit scores or ratings",
    "DebtRatioAnalysis": "Analyzing debt vs. assets or capital structure",
    "KPIRequest": "Requests for key performance indicators",
    "EntityExtraction": "Pulling specific values, entities, or fields from text",
    "AnomalyDetection": "Identifying unusual values or behavior in the data"
}


In [29]:
# ✅ Intent Handler Classes (one for each valid class)
class RevenueQuery:
    def handle(self, query):
        return "Simulated handler: Fetching revenue data..."

class RiskAssessment:
    def handle(self, query):
        return "Simulated handler: Analyzing risk factors..."

class FinancialSummary:
    def handle(self, query):
        return "Simulated handler: Summarizing financial statements..."

class InvestmentSummary:
    def handle(self, query):
        return "Simulated handler: Summarizing investment opportunity..."

class MarketVisualization:
    def handle(self, query):
        return "Simulated handler: Creating market visualizations..."

class ComparativeAnalysis:
    def handle(self, query):
        return "Simulated handler: Comparing financial periods..."

class ReportSummary:
    def handle(self, query):
        return "Simulated handler: Summarizing uploaded report..."

class ForecastRequest:
    def handle(self, query):
        return "Simulated handler: Forecasting future performance..."

class CreditScoreSummary:
    def handle(self, query):
        return "Simulated handler: Analyzing credit scores..."

class DebtRatioAnalysis:
    def handle(self, query):
        return "Simulated handler: Assessing debt ratios..."

class KPIRequest:
    def handle(self, query):
        return "Simulated handler: Retrieving KPIs..."

class EntityExtraction:
    def handle(self, query):
        return "Simulated handler: Extracting named entities..."

class AnomalyDetection:
    def handle(self, query):
        return "Simulated handler: Detecting anomalies in data..."


In [31]:
# ✅ Intent Router Mapping
intent_router = {
    "RevenueQuery": RevenueQuery(),
    "RiskAssessment": RiskAssessment(),
    "FinancialSummary": FinancialSummary(),
    "InvestmentSummary": InvestmentSummary(),
    "MarketVisualization": MarketVisualization(),
    "ComparativeAnalysis": ComparativeAnalysis(),
    "ReportSummary": ReportSummary(),
    "ForecastRequest": ForecastRequest(),
    "CreditScoreSummary": CreditScoreSummary(),
    "DebtRatioAnalysis": DebtRatioAnalysis(),
    "KPIRequest": KPIRequest(),
    "EntityExtraction": EntityExtraction(),
    "AnomalyDetection": AnomalyDetection()
}